In [43]:
import numpy as np
import sys
import csv
import re

# method = sys.argv[1]
# learning_rate = int(sys.argv[2])
# iterations = int(sys.argv[3])
# batch_size = int(sys.argv[4])
# training_data_path = sys.argv[5]
# vocabulary_path = sys.argv[6]
# testing_data_path = sys.argv[7]
# output_path = sys.argv[8]

method = 1
learning_rate = 0.1
iterations = 100
batch_size = 128
training_data_path = 'data/imdb_train.csv'
vocabulary_path = 'data/imdb_vocab'
testing_data_path = 'data/imdb_test.csv'
output_path = 'imdb_output_a.txt'


In [44]:
def load(data_path,vocab_map,num_features):
    data = []
    regex = re.compile('[^a-zA-Z ]')
    with open(data_path, 'r') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',')
        spamreader = list(spamreader)
        n = len(spamreader)
        X = np.zeros((n,num_features))
        Y = np.zeros((n,1))
        i = 0
        for row in spamreader:
            cleaned_text = regex.sub('',row[1])
            words = cleaned_text.split(' ')
            for word in words:
                X[i][vocab_map[word]] += 1
            Y[i] = int(row[0])
            i += 1
    return data

def load_data(data_path,vocab_map,num_features):
    data = load(data_path,vocab_map,num_features)
    X = data[:,1].copy()
    Y = data[:,0].copy()
    return X,Y

def load_vocab(data_path):
    vocab_map = {}
    with open(data_path, 'r') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',')        
        i = 1
        for row in spamreader:
            if row[0] not in vocab_map:
                vocab_map[row[0]] = i
                i += 1
    return vocab_map, (i-1)

In [37]:
vocab_map, m = load_vocab(vocabulary_path)

In [38]:
# print(vocab_map)
# print(m)

{'the': 1, 'and': 2, 'a': 3, 'of': 4, 'to': 5, 'is': 6, 'it': 7, 'in': 8, 'i': 9, 'this': 10, 'that': 11, 'was': 12, 'as': 13, 'for': 14, 'with': 15, 'movie': 16, 'but': 17, 'film': 18, 'on': 19, 'not': 20, 'you': 21, 'he': 22, 'are': 23, 'his': 24, 'have': 25, 'be': 26, 'one': 27, '!': 28, 'all': 29, 'at': 30, 'by': 31, 'an': 32, 'who': 33, 'they': 34, 'from': 35, 'so': 36, 'like': 37, 'there': 38, 'her': 39, 'or': 40, 'just': 41, 'about': 42, 'if': 43, 'has': 44, 'out': 45, 'what': 46, '?': 47, 'some': 48, 'good': 49, 'more': 50, 'when': 51, 'she': 52, 'very': 53, 'even': 54, 'my': 55, 'no': 56, 'up': 57, 'time': 58, 'would': 59, 'which': 60, 'only': 61, 'story': 62, 'really': 63, 'their': 64, 'see': 65, 'had': 66, 'can': 67, 'were': 68, 'me': 69, 'we': 70, 'than': 71, 'well': 72, 'much': 73, 'been': 74, 'get': 75, 'people': 76, 'will': 77, 'bad': 78, 'other': 79, 'also': 80, 'into': 81, 'do': 82, 'because': 83, 'great': 84, 'first': 85, 'how': 86, 'him': 87, 'most': 88, "don't": 89,

In [45]:
X_train,Y_train = load_data(testing_data_path,vocab_map,m)

MemoryError: 